# TP3 - Arbre et Random Forest : Prédiction du churn

Issue d'un [dataset](https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset) disponible sur Kaggle.

Le dataset pour cette séance correspond à des clients qui ont quitté ou non une banque. On souhaite savoir si le client va quitter la banque. Nous allons utiliser des arbres de décisions et des random forest pour répondre à ce problème.

## Contrôle de la qualité de donnée

Commençons par importer les données et les observer.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="whitegrid")

df = pd.read_csv("ChurnPrediction.csv")
df.head(10)

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,15574012,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,15592531,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,15656148,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,15792365,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,15592389,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


La colonne *customer_id* est unique et ne sert pas dans la prédiction. Notons que nous avons à la fois des données numérique et catégorielle.

**Consigne** : Supprimer la colonne *customer_id*

In [2]:
df.drop(columns = "customer_id", inplace = True )
df.head(10)

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


**Consigne** : En utilisant la méthode [*describe*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html), identifier s'il y a des valeurs qui paraissent aberrante dans les données numériques.

In [3]:
df.describe()

,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Pas de valeurs aberrantes à priori, tout semble cohérent

**Consigne** : Que peut-on dire des colonnes *tenure*, *products_number*, *credit_card* et *active_member* ?

Ce sont des valeurs catégoriques

**Consigne** : Calculer la proportion de déséquilibre.

In [7]:
desequilibre = len(df[df["churn"]==1]) / len(df)
desequilibre

0.2037

**Consigne** : En utilisant la fonction *agregate_column*, explorer les champs catégoriels.

In [15]:
def agregate_column(column):
    df['churn'] = pd.to_numeric(df['churn'], errors='coerce')
    grouped = df.groupby(by=column, as_index=False)['churn'].mean()
    
    return grouped

In [23]:
df["country"].value_counts()

country
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [21]:
for name in ["tenure","products_number","credit_card","active_member", "country", "gender"]:
    print(agregate_column(name))

    tenure     churn
0        0  0.230024
1        1  0.224155
2        2  0.191794
3        3  0.211100
4        4  0.205258
5        5  0.206522
6        6  0.202689
7        7  0.172179
8        8  0.192195
9        9  0.216463
10      10  0.206122
   products_number     churn
0                1  0.277144
1                2  0.075817
2                3  0.827068
3                4  1.000000
   credit_card     churn
0            0  0.208149
1            1  0.201843
   active_member     churn
0              0  0.268509
1              1  0.142691
   country     churn
0   France  0.161548
1  Germany  0.324432
2    Spain  0.166734
   gender     churn
0  Female  0.250715
1    Male  0.164559


## Préparation des données

Maintenant que l'on a *un peu* observé les données, il nous reste à les préparer pour l'entraînement.

**Consigne** : Séparer le dataset en *X* et *y*

In [20]:
X = df.drop(columns="churn")
y = df["churn"]

Puisque *X* est composé de donnée numérique comme catégorielle et que l'implémentation scikit-learn ne peut pas prendre en compte les données catégorielles, il faut les convertir.

**Consigne** : en utilisant la méthode [*get_dummies*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html), convertir avec la méthode One-Hot-Encoding les données catégorielles en données numérique. On aura prit soin de capitaliser sur les observations précédentes.

In [32]:
X = pd.get_dummies(X,columns =["country",'gender'], drop_first = True)
X.head(10)

,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,country_Germany,country_Spain,gender_Male
0,619,42,2,0.00,1,1,1,101348.88,False,False,False
1,608,41,1,83807.86,1,0,1,112542.58,False,True,False
2,502,42,8,159660.80,3,1,0,113931.57,False,False,False
3,699,39,1,0.00,2,0,0,93826.63,False,False,False
4,850,43,2,125510.82,1,1,1,79084.10,False,True,False
5,645,44,8,113755.78,2,1,0,149756.71,False,True,True
6,822,50,7,0.00,2,1,1,10062.80,False,False,True
7,376,29,4,115046.74,4,1,0,119346.88,True,False,False
8,501,44,4,142051.07,2,0,1,74940.50,False,False,True
9,684,27,2,134603.88,1,1,1,71725.73,False,False,True


## Modélisation : Arbre

On souhaite prédire le churn a partir des données que l'on vient de préparer à l'aide d'un arbre de décision. Nous allons réaliser une validation croisée pour avoir une meilleure vision des performances de l'algorithme.
Cependant, le dataset est déséquilibré, donc nous ne pouvons pas réaliser une validation croisée sans prendre en compte ce déséquilibre.

**Consigne** : Avant de régler ce problème, Construire une fonction *performance* qui prend en paramètre un vecteur *vector* et qui affiche la moyenne et l'écart-type au format suivant : *moyenne (+/- ecart-type)*. On veillera à transformer le vecteur au format *numpy* avant les traitements.

In [34]:
def cross_validation_performance(vector):
    vector = np.array(vector)
    mean_value = vector.mean()
    std_value = vector.std()
    print(f"Performance : {mean_value:.2f} (+/-{std_value:.2f})")

**Consigne** : Compléter le code suivant. Il utilise la méthode [*StratifiedKFold*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) pour entraîner un [arbre de décision](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn-tree-decisiontreeclassifier). Puis afficher les performances avec la fonction *cross_validation_performance*.

On ne souhaite plus avoir ce bloc de code systématique, nous allons donc en faire une fonction. Pour pouvoir tester plusieurs paramétrage de l'arbre, on doit être capable de lui fournir des paramètres. Voici un exemple de l'utilisation :

In [50]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score

cv = 5
folds = StratifiedKFold(n_splits=cv).split(X, y)
performances = []
accuracy = []


for (train_index, test_index) in folds:
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model_trained = DecisionTreeClassifier().fit(X_train, y_train)
    y_pred = model_trained.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    performances.append(f1)
    acc = accuracy_score(y_test,y_pred)
    accuracy.append(acc)

print("Accuracy :")
cross_validation_performance(accuracy)
print("F1 Score :")
cross_validation_performance(performances)

8000
Accuracy :
Performance : 0.79 (+/-0.01)
F1 Score :
Performance : 0.50 (+/-0.03)


In [96]:
parameters = {
    "criterion": "gini",
    "max_depth": 7,
    "min_samples_leaf": 20
}

model = DecisionTreeClassifier(**parameters)

**Consigne** : En exploitant ce fonctionnement, construire une fonction *stratified_cross_validation* qui prends en paramètre :
* *X*: le dataset des features
* *y*: le vecteur réponse
* *model*: le modèle que l'on veut tester, au format scikit-learn
* *parameters*: le dictionnaire de paramètres à transmettre à *model*
* *metric*: la métrique avec laquelle on mesure les performances de *model*, au format scikit-learn
* *cv*: le nombre de pli de la validation croisée

Elle devra renvoyer les performances sur chacun des plis.

In [93]:
def stratified_cross_validation(X, y, model, parameters, metric=f1_score, cv=3):
    folds = StratifiedKFold(n_splits=cv).split(X, y)
    performances = []
    
    
    for (train_index, test_index) in folds:
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model_trained = model(**parameters).fit(X_train, y_train)
        y_pred = model_trained.predict(X_test)
        score = metric(y_test, y_pred)
        performances.append(score)

    print(metric.__name__)
    cross_validation_performance(performances)
    

## Impact de la profondeur

On souhaiterai mesurer l'importance de la profondeur d'un arbre pour ce problème.

**Consigne** : A l'aide de la fonction précédente, répondre à la problématique avec un affichage.

In [95]:
for i in range (1,100):
    parameters['max_depth'] = i
    print("max depth:",parameters['max_depth'])
    stratified_cross_validation(X, y, DecisionTreeClassifier, parameters)


max depth: 1
f1_score
Performance : 0.00 (+/-0.00)
max depth: 2
f1_score
Performance : 0.51 (+/-0.01)
max depth: 3
f1_score
Performance : 0.45 (+/-0.05)
max depth: 4
f1_score
Performance : 0.50 (+/-0.02)
max depth: 5
f1_score
Performance : 0.55 (+/-0.02)
max depth: 6
f1_score
Performance : 0.56 (+/-0.00)
max depth: 7
f1_score
Performance : 0.58 (+/-0.01)
max depth: 8
f1_score
Performance : 0.56 (+/-0.01)
max depth: 9
f1_score
Performance : 0.56 (+/-0.01)
max depth: 10
f1_score
Performance : 0.57 (+/-0.01)
max depth: 11
f1_score
Performance : 0.57 (+/-0.00)
max depth: 12
f1_score
Performance : 0.57 (+/-0.00)
max depth: 13
f1_score
Performance : 0.57 (+/-0.00)
max depth: 14
f1_score
Performance : 0.57 (+/-0.00)
max depth: 15
f1_score
Performance : 0.57 (+/-0.00)
max depth: 16
f1_score
Performance : 0.57 (+/-0.00)
max depth: 17
f1_score
Performance : 0.57 (+/-0.00)
max depth: 18
f1_score
Performance : 0.57 (+/-0.00)
max depth: 19
f1_score
Performance : 0.57 (+/-0.00)
max depth: 20
f1_scor

Cette performance correspond en réalité au seuil 0.5. On souhaiterai être capable de trouver un seuil qui maximise le f1-score. 

## Trouver le seuil qui maximise une métrique

Pour le faire, nous allons avoir besoin de trois bases :
* Une base d'entraînement (*X_train*, *y_train*) : **entraîner** le modèle
* Une base de validation (*X_valid*, *y_valid*) : **trouver** le meilleur seuil
* Une base de test (*X_test*, *y_test*) : **tester** la performance sur des données non vues

**Consigne** : Générer les trois bases à l'aide la fonction [*train_test_split*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split), en prenant soin de conserver le même déséquilibre sur les trois bases.

In [97]:
from sklearn.model_selection import train_test_split

folds = StratifiedKFold(n_splits=2).split(X, y)
y_proba_list =[]
y_true_list =[]

for (train_index, test_index) in folds:
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    X_train, X_val, y_train, y_val = train_test_split(X.iloc[train_index], y.iloc[train_index], test_size=0.2, random_state=42, stratify = y.iloc[train_index])
    X_test, y_test = X.iloc[test_index], y.iloc[test_index]
    y_true_list.append(y_val)
    model_trained = DecisionTreeClassifier(**parameters).fit(X_train, y_train)
    y_proba = model_trained.predict_proba(X_val)
    y_proba_list.append(y_proba)
    y_true_list.append(y_val)
    

**Consigne** : Entraîner un arbre puis prédire les probabilités d'être de la classe d'intérêt pour le dataset de validation. Les stocker dans une variable *y_proba*.

In [98]:
print(y_proba_list)

[array([[0.67741935, 0.32258065],
       [1.        , 0.        ],
       [0.91836735, 0.08163265],
       ...,
       [0.859375  , 0.140625  ],
       [0.55757576, 0.44242424],
       [0.65      , 0.35      ]]), array([[1.        , 0.        ],
       [0.47619048, 0.52380952],
       [0.61764706, 0.38235294],
       ...,
       [1.        , 0.        ],
       [0.95714286, 0.04285714],
       [0.57894737, 0.42105263]])]


**Consigne** : Construire une fonction *find_best_treshold* qui prends en paramètre :
* *y_true* : vecteur des classes attendues
* *y_proba* : vecteur de probabilité estimé des classes
* *metric* : métrique à optimiser, au format scikit-learn
Elle revoit la meilleure performance et le meilleur seuil pour la métrique sélectionnée

In [126]:
def find_best_threshold(y_true, y_proba, metric):
    thresholds = np.linspace(0,1,100)
    f1 = 0
    seuil = 0
    for threshold in set(y_proba[:,1]):
        y_pred = y_proba[:,1] > threshold
        score = metric(y_true, y_pred)
        if score > f1:
            f1 = score
            seuil = threshold
    return seuil, f1

**Consigne** : Utiliser la fonction *find_best_threshold* sur le jeu de validation, et comparer avec la performance obtenue sur le jeu de test.

In [127]:
for i in range(len(y_proba_list)):
    print(find_best_threshold(y_true_list[i],y_proba_list[i],f1_score))
    

(0.35, 0.6175771971496438)
(0.36363636363636365, 0.6024691358024692)


**Consigne** : Reprendre la fonction *stratified_cross_validation* et la modifier pour afficher la meilleure performance que l'on puisse obtenir, avec en plus la valeur du seuil.

In [128]:
def stratified_cross_validation_seuil(X, y, model, parameters, metric=f1_score, cv=3):
    folds = StratifiedKFold(n_splits=cv).split(X, y)
    y_proba_list =[]
    y_true_list =[]
    scores = []
    thresholds = []
    

    for (train_index, test_index) in folds:
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        X_train, X_val, y_train, y_val = train_test_split(X.iloc[train_index], y.iloc[train_index], test_size=0.2, random_state=42, stratify = y.iloc[train_index])
        X_test, y_test = X.iloc[test_index], y.iloc[test_index]
        y_true_list.append(y_val)
        model_trained = DecisionTreeClassifier(**parameters).fit(X_train, y_train)
        y_proba = model_trained.predict_proba(X_val)
        threshold, score = find_best_threshold(y_val,y_proba,f1_score)
        
        y_pred = model_trained.predict_proba(X_test)
        y_pred = y_pred[:,1] > threshold
        score = metric(y_test, y_pred)
        scores.append(score)
    print("f1-score:", np.mean(scores))
        
stratified_cross_validation_seuil(X, y, model, parameters, metric=f1_score, cv=5)
    
    
    

f1-score: 0.6042860150944578


## Impact de la profondeur : le retour

Maintenant que l'on sait obtenir la meilleur version de chaque algorithme, on souhaite mesurer un peu mieux l'impact de la profondeur.

**Consigne** : A l'aide de la fonction précédente, répondre à la problématique avec un affichage.

In [129]:
for i in range (1,100):
    parameters['max_depth'] = i
    print("max depth:",parameters['max_depth'])
    stratified_cross_validation(X, y, DecisionTreeClassifier, parameters)


max depth: 1
f1-score: 0.48868860267827546
max depth: 2
f1-score: 0.5126169591842874
max depth: 3
f1-score: 0.5390218014406685
max depth: 4
f1-score: 0.552392253957218
max depth: 5
f1-score: 0.5759265477104119
max depth: 6
f1-score: 0.5765872088079479
max depth: 7
f1-score: 0.5787258668702613
max depth: 8
f1-score: 0.5844059059602554
max depth: 9
f1-score: 0.5893050709175917
max depth: 10
f1-score: 0.5834173478054265
max depth: 11
f1-score: 0.5853884772803508
max depth: 12
f1-score: 0.5853884772803508
max depth: 13
f1-score: 0.5844177506703414
max depth: 14
f1-score: 0.5853884772803508
max depth: 15
f1-score: 0.5838975467587599
max depth: 16
f1-score: 0.5853884772803508
max depth: 17
f1-score: 0.5849082541179577
max depth: 18
f1-score: 0.5854689292647898
max depth: 19
f1-score: 0.5844177506703414
max depth: 20
f1-score: 0.5849082541179577
max depth: 21
f1-score: 0.5853884772803508
max depth: 22
f1-score: 0.5848642789132529
max depth: 23
f1-score: 0.5839779987431989
max depth: 24
f1-sco

On souhaiterai avoir une représentation visuelle de cet affichage. Pour ce faire, on définit la fonction suivante.

In [115]:
def plot_performance(parameters, performances, color=None, label=None,confidence=3):
    if color is None: color=sns.color_palette()[0]
    if label is None: label=""
        
    mean = [performance.mean() for performance in performances]
    deviation = [performance.std() for performance in performances]
    
    mean, deviation = np.array(mean), np.array(deviation)
    
    plt.fill_between(parameters, mean - confidence*deviation, mean + confidence*deviation, alpha=0.15, color=color)
    plt.plot(parameters, mean, 'o-', color=color, label=label)

**Consigne** : en reprenant la question précédente (en adaptant), et en utilisant la fonction *plot_performance*, montrer visuellement l'impact de la profondeur sur la performance.

## Et la Random Forest ?

On s'intéresse maintenant à la Random Forest. On souhaite mesurer la même chose que pour l'arbre.

**Consigne** : reproduire la même étude, mais avec une Random Forest de 50 arbres.

**Consigne** : Afficher sur le même graphique, avec une légende, les performances pour un arbre et pour une Random Forest.

## Et maintenant ?

Il existe d'autres hyperparamètres important dans ces modèles. Reproduire les études, et comparer les performances entre arbres et Random Forest voire Extra-Trees.